In [1]:
import pandas as pd
import numpy as np

In [2]:
class data(object):
    def __init__(self,file,csv_indicator):
        self.file = file
        self.csv_indicator = csv_indicator
        self._load_data()
        
    def _load_data(self):
        if self.csv_indicator==1:
            self.df = pd.read_csv(self.file,skiprows=1).drop('ID',1)
        else:
            self.df = pd.read_excel(self.file,skiprows=1).drop('ID',1)
        
    #column X3 or education should only have values 1-4. 
    #take out the 5,6,7 values that show up
    #what should our methodology be for replacing these values?
    #random selection?  Ask Tao what we should do with these values.
    #i think randomly selecting a value (1-4) based on actual distribution of column makes sense
    #to do this calculate length of distribution w/ acceptable values
    #generate random number inside length of that distribution
    #index acceptable values by random number you generated
    def _fix_education_values(self):
        def replace_values(x):
            if x > 4:
                return 4
            else:
                return x
            
        self.df['EDUCATION'] = [replace_values(x) for x in self.df.EDUCATION]
        
    
    #should have 1 (married),2 (single),3 (others)
    #replace all 0's with 3 as they will fall under the category "others"
    def _fix_marital_status(self):
        def fix_numbers(x):
            if x == 0:
                return 3
            else:
                return x
        self.df['MARRIAGE'] = [fix_numbers(x) for x in self.df.MARRIAGE]
       
        
    def _fix_late_pay_status(self,include_columns):
        for column in include_columns:
            def max_(x):
                return max(x,0)
            self.df[column] = [max_(x) for x in self.df[column]]
        #columns 6-11
        #anything negative should be switched to 0. This indicates they have paid on time and in some cases two months early.
        #columns 12-17
        #this could also be used on these columns. Anything negative indicates they have overpayed. Turn the columns into...
        #...money due so replace all negative values.
        
    def _fix_outliers(self,exclude_columns=None):
        def get_outliers(self,value,lower,upper):
            if value >= upper + 1.5*upper:
                return upper
            elif value <= lower - 1.5*lower:
                return lower
            else:
                return value
    
        if exclude_columns==None:
            unique_cols = self.df.columns
        else:
            unique_cols = self.df.columns.drop(exclude_columns,1)
        for i in unique_cols:
            lower = self.df[i].quantile(0.225)
            upper = self.df[i].quantile(0.775)
            self.df[i]= [get_outliers(self,value,lower,upper) 
                         for value,lower,upper in 
                         zip(self.df[i],
                             [lower for x in range(0,len(self.df))],
                             [upper for y in range(0,len(self.df))]
                            )
                        ]
            
    
    #def _normalize_stuff(self,exclude_columns=None):
        #how do we make up a set of rules that checks for skewness and applies a certain function to the distribution
        #if that skewness still isn't within acceptable bounds try different function
    
    #scale all the columns (0-1)
    def _scale_stuff(self,exclude_columns=None):
        from sklearn.preprocessing import MinMaxScaler
        min_max_scaler = MinMaxScaler()
        if exclude_columns==None:
            unique_cols = self.df.columns
        else:
            unique_cols = self.df.columns.drop(exclude_columns,1)
        
        for col in unique_cols:
            self.df[col] = pd.Series(min_max_scaler.fit_transform(np.array(self.df[col]).reshape(-1,1)).reshape(1,len(self.df[col]))[0])
            
    def _calculate_z_scores(self,exclude_columns=None):
        if exclude_columns== None:
            unique_cols = self.df.columns
        else:
            unique_cols = self.df.columns.drop(exclude_columns,1)
        for col in unique_cols:
            self.df[col] = (self.df[col] - self.df[col].mean())/self.df[col].std(ddof=0)
    
    #def _calculate_x_features():
        
    #def _calculate_target_variables():
        
    #def _bin_column_values(column_name)
        

In [5]:
F = data('default_of_credit_card_clients.xlsx',0)
H = data('default_of_credit_card_clients.xlsx',0)
#H = data('default_of_credit_card_clients.xlsx',0)

categorical = ['SEX','EDUCATION','MARRIAGE','AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','default payment next month']

F._fix_outliers(categorical)

#is the fix outliers function changing the dataframe?
F.df.head(15)==H.df.head(15)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True


##### F = data('default_of_credit_card_clients.xlsx',0)
F._fix_outliers(categorical)
F._calculate_z_scores(categorical)
F.df

In [3]:
#NORMAL WORK FLOW EXAMPLE BELOW

#include_columns = [Columns X6-X11]  
include_columns = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
#include_columns1 = [Columns X12-X17]
include_columns1 = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
#categorical = [Categorical Columns]
categorical = ['SEX','EDUCATION','MARRIAGE','AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','default payment next month']

#initialize object
F = data('default_of_credit_card_clients.xlsx',0)

#fix known bad stuff, maybe call these in the initialize function
F._fix_marital_status()
F._fix_late_pay_status(include_columns)
F._fix_late_pay_status(include_columns1)
F._fix_education_values() 
       
    
#THEN IN ANY ORDER, TRY DIFFERENT ORDERS OUT TO SEE WHAT WORKS BEST
F._fix_outliers(categorical)
#F._normalize_stuff(categorical)
F._scale_stuff(categorical)
F._calculate_z_scores(categorical)

#use above datdaframe to then test which order produces best modeling performance
#take modeling code from evaluation code in competition1 (xgboost model)

In [11]:
y = F.df['default payment next month']
X = F.df.drop('default payment next month',1)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
scores = []
count=0
for i in range(0,2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    model = XGBClassifier(learning_rate = 0.1,
                      n_estimators=1000,
                      max_depth=10,
                      min_child_weight=1,
                      gamma=0.5,
                      reg_alpha=0.05,
                      colsample_bytree=0.8,
                      objective='binary:logistic',
                      scal_pos_weight=1,
                      seed=27)

    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    scores.append(accuracy)
    count = count + 1
    print(count)

C:\Users\e333775\AppData\Local\Continuum\anaconda3_n\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


1
2


C:\Users\e333775\AppData\Local\Continuum\anaconda3_n\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
print(scores)
#only 20% of people defaulted.
#If we guessed "not default" everytime we'd be about 80% correct so the above results actually kind of suck
sum(F.df['default payment next month'])/len(F.df['default payment next month'])


[0.815, 0.8116666666666666]


0.2212

In [17]:
#things to do
    #try messing around with the order of the data cleaning functions
    #try creating new columns that may have predictive value based on data we have
    #try gridsearchcv to optimize parameters of XGBoost
    #Run on more than 2 trials for each different dataset w/ gridsearch to see which performs the best
    #figure out how to define the normalization data cleaning method
    #ask Tao about other shit we could do